In [1]:
from __future__ import print_function

import math
import re
import os
import sys
import json
import shutil
import string
import datetime
from IPython import display
import numpy as np
import pandas as pd
import cPickle
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, roc_auc_score, log_loss, precision_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.learning_curve import learning_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


# Setup environment
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# present directory
cwd = os.getcwd()

# portal directory
portal_dir = '/apps/rpauser/portal/rpaportal/app/static/src/assets/ml/'
#portal_dir = '/tmp/'

# Class for dumping o/p as json
class model_output(object):
  def __init__(self, Directory, Date, Id, Model, AUC, Accuracy, LogLoss, Precision, Input, Prediction, Details):
    self.Directory = cwd + '/' + Directory
    self.Date = str(Date)
    self.Id = Id
    self.Model = Model
    self.Accuracy = Accuracy
    self.AUC = AUC
    self.LogLoss = LogLoss
    self.Precision = Precision
    self.Input = Input
    self.Prediction = Prediction
    self.Details = Details

  def set_value(self, name, value):
    if name == "AUC":
      self.AUC = str("%0.2f" % value)
    elif name == "Accuracy":
      self.Accuracy = str(("%0.2f" % value) * 100) + '%'
    elif name == "LogLoss":
      self.LogLoss = str("%0.2f" % value)
    elif name == "Precision":
      self.Precision = str(("%0.2f" % value) * 100) + '%'
    elif name == "Input":
      self.Input = value
    elif name == "Prediction":
      self.Prediction = value
    elif name == "Details":
      self.Details = value
    else:
      print('output value not supported.')


  def dump_json(self):
   f = open(self.Directory + '/model_output.json', 'w')
   f.write(json.dumps(self, default=jdefault, indent=4, sort_keys=True))
   f.close()


# Utility for json dumping
def jdefault(o):
  if isinstance(o, set):
    return list(o)
  return o.__dict__

# Print and log into output folder
def log(outdir, desc, value):
  print(desc)
  print(str(value))
  filename = outdir + '/log.txt'
  f = open(filename, 'a')
  f.write(desc)
  f.write(str(value))
  f.write('\n')
  f.close()
  # Copy for portal usage
  shutil.copy2(filename, portal_dir + filename.replace("/", ""))

# Utility function to setup directory to persist output
def prep_outdir(id_):
   now = datetime.datetime.now()
   directory =  os.path.join('output', now.strftime("%Y%m%d%H%M%S")+'_'+id_)
   if not os.path.exists(directory):
     os.makedirs(directory)
   return now, directory

# Utility function to save hyperparameters
def save_params(now, directory, id_, model):
   filename = directory + '/log.txt'
   f = open(filename, 'a')
   f.write('Time=' + str(now))
   f.write('\nId=' + id_)
   f.write('\nModel=' + model )
   f.write('\nDetails=' + filename.replace("/", ""))
   f.close()
   # Copy for portal usage
   shutil.copy2(filename, portal_dir + filename.replace("/", ""))

   mop = model_output(directory, now, id_, model,  '', '', '', '', '', '', filename.replace("/", ""))
   mop.dump_json()
   return mop

# Utility function to save prediction
def save_prediction(directory, mop, prediction):
   filename = directory + '/prediction.csv'
   prediction.to_csv(filename, sep=",", encoding='utf-8')
   # Copy for portal usage
   shutil.copy2(filename, portal_dir + filename.replace("/", ""))
   mop.set_value('Prediction', filename.replace("/", ""))
   mop.dump_json()


# Convert string to a list
def convert_to_list(text):
  list_data = []
  words = text.split()
  for word in words:
    list_data.append(word)
  return list_data

# Remove punctuations, stop words, stemming
def clean_text(text):
  text = re.sub(r"\r", "", text)
  text = re.sub(r"\n", "", text)
  text = text.strip(' ')
  return text

# Preprocess data
def preprocess_data(dataframe):
  # Create binary feature
  df = dataframe
  mapping = {'Acknowledgement': 1, 'Other': 0, 'Partial_Answer': 0, 'Full_Answer' :0}
  df = df.replace({'label': mapping})
  dataframe["ack"] = df["label"]

  df = dataframe
  mapping = {'Acknowledgement': 0, 'Other': 1, 'Partial_Answer': 0, 'Full_Answer' :0}
  df = df.replace({'label': mapping})
  dataframe["other"] = df["label"]

  df = dataframe
  mapping = {'Acknowledgement': 0, 'Other': 0, 'Partial_Answer': 1, 'Full_Answer' :0}
  df = df.replace({'label': mapping})
  dataframe["partial"] = df["label"]

  df = dataframe
  mapping = {'Acknowledgement': 0, 'Other': 0, 'Partial_Answer': 0, 'Full_Answer' :1}
  df = df.replace({'label': mapping})
  dataframe["full"] = df["label"]

  dataframe['msg'] = dataframe['msg'].map(lambda x : clean_text(x))
  return dataframe

# Read the input data for training and validation
def read_input(now, outdir):
  # Read input data
  df = pd.read_csv("input/training.csv", sep=",", encoding='latin-1')

  log(outdir, 'Input dataframe before preprocessing', df)
  log(outdir, 'Input dataframe describe before preprocessing', df.describe())

  # Randomize and cleanup the data
  df = df.reindex(np.random.permutation(df.index))
  df = preprocess_data(df)

  log(outdir, 'Input dataframe after preprocessing', df)
  log(outdir, 'Input dataframe describe after preprocessing', df.describe())

  return df


# Pipeline Factory
def get_pipeline(model):
  if model == "MultinomialNB":
    pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
  elif model == "LinearSVC":
    pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
  elif model == "LogisticRegression":
    pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
  else:
    print('Model not supported')
  return pipeline

# Params Factory
def get_params(model):
  if model == "MultinomialNB":
    params = {
      'tfidf__use_idf': (True, False),
    }
  elif model == "LinearSVC":
    params = {
      'tfidf__use_idf': (True, False),
    }
  elif model == "SGDClassifier":
    params = '' 
  elif model == "LogisticRegression":
    params = {
      'tfidf__use_idf': (True, False),
    }
  else:
    print('Model not supported')
  return params

# Train the model 
def train_validate_model(model, mop, df, df_test, now, outdir):

  categories = ['ack', 'other', 'partial', 'full']
  train, vald = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)
  X_train = train.msg
  X_vald = vald.msg
  X_test = df_test.msg
  log(outdir, 'Training data size: ', str(X_train.shape[0]))
  log(outdir, 'Validate data size: ', str(X_vald.shape[0]))
  log(outdir, 'Test data size: ', str(X_test.shape[0]))

  # Define a pipeline combining a text feature extractor with multi lable classifier
  pipeline = get_pipeline(model)

  result = df_test
  # Train the model, minimise loss
  for category in categories:
    log(outdir, '... Processing ...', category)

    grid = GridSearchCV(
      get_pipeline(model),  # pipeline
      get_params(model),    # parameters to tune via cross validation
      refit=True,           # fit using all available data at the end, on the best found param combination
      n_jobs=-1,            # number of cores to use for parallelization; -1 for "all cores"
      scoring='accuracy',   # what score are we optimizing?
      cv=StratifiedKFold(train[category], n_folds=5),  # what type of cross validation to use
    )
    # train the model 
    model_detector = grid.fit(X_train, train[category])

    # compute the validation accuracy
    prediction = model_detector.predict(X_vald)

    mop.set_value('Accuracy', accuracy_score(vald[category], prediction))
    log(outdir, 'accuracy is: ', accuracy_score(vald[category], prediction))

    mop.set_value('AUC', roc_auc_score(vald[category], prediction))
    log(outdir, 'roc_auc is: ', roc_auc_score(vald[category], prediction))

    mop.set_value('LogLoss', log_loss(vald[category], prediction))
    log(outdir, 'log_loss is: ', log_loss(vald[category], prediction))

    mop.set_value('Precision', precision_score(vald[category], prediction))
    log(outdir, 'precision is: ', precision_score(vald[category], prediction))

    mop.dump_json()

    log(outdir, 'Confusion Matrix', confusion_matrix(vald[category], prediction))
    log(outdir, 'Classification Report', classification_report(vald[category], prediction))

    # predict the testing data
    prediction = model_detector.predict(X_test)
    prediction_ = pd.DataFrame(prediction)
    prediction_.columns = prediction_.columns.astype(str)
    prediction_.rename(columns={"0": category}, inplace=True)
    prediction_.reset_index(drop=True, inplace=True)
    result.reset_index(drop=True, inplace=True)
    result = pd.concat([result, prediction_], axis=1)

    # store the spam detector to disk after training
    filename = outdir + '/' + category + '-model.pkl'
    with open(filename, 'wb') as out:
      cPickle.dump(model_detector, out)
      log(outdir, 'Model Saved', filename)


  log(outdir, "Prediction: ", result)
  save_prediction(outdir, mop, result)


# Testing of the model
def read_test(now, outdir):

  # Read input data
  df_test = pd.read_csv("input/test.csv", sep="\t", encoding='latin-1')

  # Randomize and cleanup the data
  #test = preprocess_data(test)
  df_test['msg'] = df_test['msg'].map(lambda x : clean_text(x))

  log(outdir, 'Test dataframe describe after preprocessing', df_test)
  log(outdir, 'Test dataframe describe after preprocessing', df_test.describe())

  return df_test


# Training validation  and testing of the model
def train_validate_test():

  hyperparameters_dataframe = pd.read_csv("input/hyperparams.csv", sep=",")
  print('Hyperparameters')
  print(hyperparameters_dataframe)
  for index, row in hyperparameters_dataframe.iterrows():

    id_ = row[0]
    modl = row[1]

    # Read Data, normalize, preprocess features, view
    print('\n\n ************ Training new model - ' + id_ + ' *************')

    # Creates an outputdir for the model
    now, outdir = prep_outdir(id_)

    # saving all config
    mop = save_params(now, outdir, id_, modl)

    # Reading input
    df = read_input(now, outdir)

    # Reading test
    df_test = read_test(now, outdir)

    # Train Model
    train_validate_model(modl, mop, df, df_test, now, outdir)

    # Test Model
    #test(pipeline, mop, outdir)


# Main
train_validate_test()


ModuleNotFoundError: No module named 'cPickle'